In [1]:
import os
import pandas as pd
from transformers import BertTokenizer
import six
import re
import json
import numpy as np
import time
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from seqeval.metrics import classification_report as classification_report_seqeval

import tensorflow as tf
from tensorflow import keras
from transformers import TFElectraModel, TFBertModel
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, Activation, Lambda, Input
from tf2crf import CRF, ModelWithCRFLoss

/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/tensorflow_addons/utils/ensure_tf_install.py:67: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.4.0 and strictly below 2.7.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.3.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  UserWarning,


In [2]:
# tokenizer = BertTokenizer.from_pretrained('../../electra/pretrained_model/nscc_mecab_base/vocab.txt', do_lower_case=False)
tokenizer = BertTokenizer.from_pretrained('vocab.txt', do_lower_case=False)

Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated


In [3]:
df_datas = pd.read_csv('NER_ids_label_rev_index_1102.csv')

x_data = df_datas['summary'].tolist()
# x_data = df_datas['text'].tolist()[:int(len(df_datas)/4)]
y_data = df_datas['NER_ids'].tolist()
for i in range(len(y_data)):
    y_data[i] = [int(i) for i in (y_data[i][1:-1].split())]
str_y_data = df_datas['NER_label']

In [39]:
str_train_x, str_test_x, train_y, test_y = train_test_split(x_data, y_data, test_size=0.3, random_state=72)

print(f"Train : {len(str_train_x)} / Test : {len(str_test_x)}")

Train : 86537 / Test : 37088


In [5]:
MAX_LENGTH = 256

In [6]:
np_train_y = np.array(train_y)
np_test_y = np.array(test_y)

In [7]:
with open("ner_to_index_rev_1030.json", 'rb') as f:
    ner_to_index = json.load(f)
index_to_ner = {}
for key, value in ner_to_index.items():
    index_to_ner[value] = key

In [8]:
BATCH_SIZE = 32
vocab_size = len(tokenizer)
tag_size = len(ner_to_index)

In [9]:
with tf.device('/device:GPU:1'):
    new_model = tf.keras.models.load_model('models/electra_NER_ids_rev_21.11.03-2_model')
    new_model.summary()

Model: "model_with_crf_loss"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
functional_1 (Functional)    ((None, 256), (None, 256, 110880518 
_________________________________________________________________
accuracy (Accuracy)          multiple                  2         
_________________________________________________________________
loss (Mean)                  multiple                  2         
Total params: 110,880,522
Trainable params: 110,880,516
Non-trainable params: 6
_________________________________________________________________


In [10]:
encode_test_data = []
input_ids =[]
attention_masks =[]
token_type_ids =[]
for line in tqdm(str_test_x):
    encoded_dict = tokenizer.encode_plus(line, \
                                         add_special_tokens = True,\
                                         pad_to_max_length=True,\
                                         max_length=MAX_LENGTH, 
                                        return_attention_mask=True,
                                        truncation = True)

    input_id=encoded_dict['input_ids']
    attention_mask=encoded_dict['attention_mask']
    token_type_id = encoded_dict['token_type_ids']
    
    input_ids.append(input_id)
    attention_masks.append(attention_mask)
    token_type_ids.append(token_type_id)

test_input_ids=np.array(input_ids, dtype=int)
test_attention_masks=np.array(attention_masks, dtype=int)
test_token_type_ids=np.array(token_type_ids, dtype=int)
test_x=(test_input_ids, test_attention_masks, test_token_type_ids)

  0%|          | 0/37088 [00:00<?, ?it/s]/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/transformers/tokenization_utils_base.py:2155: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
100%|██████████| 37088/37088 [01:46<00:00, 348.85it/s]


In [11]:
with tf.device('/device:GPU:1'):
    y_pred = new_model.predict(test_x)

In [12]:
# type(y_pred)
list_y_pred = list(y_pred)
list_test_input_ids = list(test_input_ids)

In [36]:
(list_test_input_ids[0][0])
# (list_y_pred[0][0])
# (test_y[0][0])
# (str_test_x[0])

2

In [13]:
# with tf.device('/device:GPU:1'):
#     score_his = new_model.evaluate(test_x, test_x)
#     print(score_his)

In [14]:
type(test_y[0][0])
type(test_x[0][0][0])
# cast_y = tf.cast(test_y, dtype='int32')

numpy.int64

In [40]:
test_y[0]
cast_y[0]

<tf.Tensor: shape=(256,), dtype=int32, numpy=
array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)>

In [15]:
target_names = []
none_target_names = ['[CLS]', '[SEP]', '[PAD]', '[MASK]', 'O']
for ner_tag in ner_to_index.keys():
    if ner_tag in none_target_names:
        continue
    else:
        target_names.append(ner_tag)
#     target_names.append(ner_tag)
    
print("len test_y : ", len(test_y))
print("len y_pred : ", len(y_pred))
print("len str_test_x : ", len(str_test_x))
print("len test_x[0](ids) : ", len(test_x[0]))
print("target names ; ", target_names)
print("tag size : ", tag_size)
print("code 종류 : ", (tag_size-5)//2)

report_test_y = [] # classification report용 test_y(실제값) 리스트를 만들어줌 (태깅 코드를 하나씩 리스트로 추가해줌)
report_test_y_code = []
report_y_pred = [] # classification report용 y_pred(예측값) 리스트를 만들어줌 (태깅 코드를 하나씩 리스트로 추가해줌)
report_y_pred_code = []
for test_ids_list, pred_ids_list in tqdm(zip(np_test_y, y_pred), total = len(test_y)):
    test_y_code = []
    y_pred_code = []
    for test_ids, pred_ids in zip(test_ids_list, pred_ids_list):
        report_test_y.append(test_ids)
        report_y_pred.append(pred_ids)
        test_code = index_to_ner[test_ids].replace('B▁', 'B-').replace('I▁', 'I-').replace('[CLS]', 'O').replace('[SEP]', 'O').replace('[PAD]', 'O')
        pred_code = index_to_ner[pred_ids].replace('B▁', 'B-').replace('I▁', 'I-').replace('[CLS]', 'O').replace('[SEP]', 'O').replace('[PAD]', 'O')
        if test_code == '[CLS]' or test_code == '[SEP]' or test_code == '[PAD]':
            continue
        if not test_code:
            print("test : ", index_to_ner[test_ids])
            print("pred : ", index_to_ner[pred_ids])
        test_y_code.append(test_code)
        y_pred_code.append(pred_code)
    report_test_y_code.append(test_y_code)
    report_y_pred_code.append(y_pred_code)
        
label_index_to_print = list(range(5, tag_size))  # ner label indice except '[CLS]', '[SEP]', '[PAD]', '[MASK]' and 'O' tag
# label_index_to_print = list(range(tag_size))  # ner label indice except '[CLS]', '[SEP]', '[PAD]', '[MASK]' and 'O' tag
# print(classification_report(y_true=report_test_y, y_pred=report_y_pred, target_names=target_names, labels=label_index_to_print, digits=4))

print("len report_y_pred : ", len(report_y_pred))
print("len report_test_y : ", len(report_test_y))
print("type y_pred : ", type(y_pred))
print("type y_pred[0] : ", type(y_pred[0]))

len test_y :  37088
len y_pred :  37088
len str_test_x :  37088
len test_x[0](ids) :  37088
target names ;  ['B▁HB01', 'I▁HB01', 'B▁SD07', 'I▁SD07', 'B▁HD05', 'I▁HD05', 'B▁NA05', 'I▁NA05', 'B▁LA07', 'I▁LA07', 'B▁SC02', 'I▁SC02', 'B▁EE04', 'I▁EE04', 'B▁LC01', 'I▁LC01', 'B▁SB11', 'I▁SB11', 'B▁EH10', 'I▁EH10', 'B▁LB07', 'I▁LB07', 'B▁ND11', 'I▁ND11', 'B▁SF04', 'I▁SF04', 'B▁NC09', 'I▁NC09', 'B▁HE05', 'I▁HE05', 'B▁NA10', 'I▁NA10', 'B▁NB06', 'I▁NB06', 'B▁EG03', 'I▁EG03', 'B▁SD04', 'I▁SD04', 'B▁LC09', 'I▁LC09', 'B▁EC09', 'I▁EC09', 'B▁EB05', 'I▁EB05', 'B▁EI07', 'I▁EI07', 'B▁ND06', 'I▁ND06', 'B▁SH03', 'I▁SH03', 'B▁ND10', 'I▁ND10', 'B▁LC15', 'I▁LC15', 'B▁HB09', 'I▁HB09', 'B▁LC13', 'I▁LC13', 'B▁LB19', 'I▁LB19', 'B▁SB01', 'I▁SB01', 'B▁SD08', 'I▁SD08', 'B▁SF01', 'I▁SF01', 'B▁EG02', 'I▁EG02', 'B▁EG10', 'I▁EG10', 'B▁SC12', 'I▁SC12', 'B▁ND13', 'I▁ND13', 'B▁SC07', 'I▁SC07', 'B▁HE03', 'I▁HE03', 'B▁HB03', 'I▁HB03', 'B▁SD03', 'I▁SD03', 'B▁SB09', 'I▁SB09', 'B▁SC01', 'I▁SC01', 'B▁SA07', 'I▁SA07', 'B▁SA03', '

100%|██████████| 37088/37088 [00:48<00:00, 764.43it/s]

len report_y_pred :  9494528
len report_test_y :  9494528
type y_pred :  <class 'numpy.ndarray'>
type y_pred[0] :  <class 'numpy.ndarray'>


In [40]:
# predict_file_name = 'predict_NER_electra_ids_rev_21.11.03-2.csv'
predict_file_name = 'a1105.csv'
# df_predict = pd.DataFrame({'input' : str_test_x, 'target' : test_y, 'predict' : list_y_pred})
df_predict = pd.DataFrame({'input' : str_test_x, 'encode_input' : list_test_input_ids, 'target' : test_y, 'predict' : list_y_pred})
df_predict.to_csv(predict_file_name, index=False)

In [41]:
pd.read_csv(predict_file_name)

,input,encode_input,target,predict
0,작물별 토양식물체 비소 전달함수transfer function 도출 작물의 가식부...,[ 2 19717 4956 5803 2392 15412 4798 73...,"[0, 4, 471, 4, 4, 331, 159, 160, 4, 4, 4, 4, 4...",[ 0 4 471 4 4 331 159 160 4 4 4 ...
1,CTIF과 DDX19의 상호작용을 통한 CBP8020의존적 번역의 이해,[ 2 6584 7225 2090 24737 2167 29055 76...,"[0, 31, 4, 4, 4, 4, 4, 4, 4, 275, 276, 276, 4,...",[ 0 31 4 4 4 4 4 4 4 275 276 2...
2,O 전처리용 신선농산물구근류의 수확후 환경적 요인이 갈변을 주체로 하는 품질저하에 ...,[ 2 37 6175 2125 11488 15362 2079 20...,"[0, 4, 329, 4, 233, 234, 4, 4, 4, 105, 325, 4,...",[ 0 4 329 4 233 234 4 4 4 105 325 ...
3,주관기관일진복합소재주 최적의 비무게kgL를 갖는 내압용기 개발을 위해 복합재료 보...,[ 2 5179 2256 2087 17391 2240 5085 9...,"[0, 4, 4, 4, 4, 4, 107, 4, 4, 4, 4, 4, 4, 585,...",[ 0 4 4 4 4 4 107 4 4 4 4 ...
4,정성적 접근 주거환경 지표 간 상대적 중요도를 도출 근린 생활권의 물리적 규모 차...,[ 2 13623 4725 7895 4142 5001 78 188...,"[0, 4, 539, 163, 164, 69, 4, 4, 4, 4, 4, 4, 38...",[ 0 4 539 163 164 69 4 4 4 4 4 ...
...,...,...,...,...
37083,JMJD6의 히스톤 탈메틸화 효소로서의 기능 연구 JMJD6의 타겟 유전자를 조사하...,[ 2 32 2135 2586 2139 16399 13207 17...,"[0, 4, 4, 4, 4, 4, 19, 20, 20, 20, 4, 19, 20, ...",[ 0 4 4 4 4 4 385 275 276 27 4 ...
37084,수출 토마토 ICT 활용농가의 경영실태 조사2017년 연구대상 토마토 수출 경영체...,[ 2 4412 10781 5745 4021 14859 6312 100...,"[0, 285, 287, 561, 562, 4, 287, 288, 313, 4, 4...",[ 0 285 287 493 43 4 107 4 313 4 4 2...
37085,13세부11 다중초점TSOM 이미지 획득 기술 TSOM 결합된 결상조명AF 광학 모...,[ 2 6920 5600 7561 4789 16346 7877 109...,"[0, 4, 4, 4, 485, 486, 4, 4, 477, 19, 375, 4, ...",[ 0 4 4 4 485 486 4 4 477 19 375 ...
37086,일반 결핵의 영상 소견은 비교적 잘 알려져 있지만 다제 내성 결핵과 광범위 약제 내...,[ 2 4629 8803 2105 4293 19298 2160 101...,"[0, 519, 31, 4, 79, 4, 4, 4, 4, 4, 4, 501, 502...",[ 0 519 31 4 79 4 4 4 4 4 4 ...


In [ ]:
# report_test_y = [] # classification report용 test_y(실제값) 리스트를 만들어줌 (태깅 코드를 하나씩 리스트로 추가해줌)
# report_test_y_code = []
# for ids_list in test_y:
#     test_y_code = []
#     for ids in ids_list:
#         report_test_y.append(ids)
#         test_y_code.append(index_to_ner[ids].replace('B▁', 'B-').replace('I▁', 'I-'))
#     report_test_y_code.append(test_y_code)
        
# report_y_pred = [] # classification report용 y_pred(예측값) 리스트를 만들어줌 (태깅 코드를 하나씩 리스트로 추가해줌)
# report_y_pred_code = []
# for ids_list in y_pred:
#     y_pred_code = []
#     for ids in ids_list:
#         report_y_pred.append(ids)
#         y_pred_code.append(index_to_ner[ids].replace('B▁', 'B-').replace('I▁', 'I-'))
#     report_y_pred_code.append(y_pred_code)

In [54]:
report_y_pred_code[0]

['O',
 'O',
 'B-ND01',
 'O',
 'O',
 'B-NC04',
 'B-ED09',
 'I-ED09',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-NC04',
 'O',
 'O',
 'B-EH07',
 'O',
 'O',
 'B-LC03',
 'O',
 'O',
 'O',
 'B-ND01',
 'B-ND11',
 'I-ND11',
 'O',
 'O',
 'B-ND12',
 'O',
 'B-NB06',
 'O',
 'O',
 'O',
 'B-NB06',
 'B-LB02',
 'I-LB02',
 'O',
 'B-LC04',
 'B-ND11',
 'B-SC09',
 'O',
 'O',
 'B-NC04',
 'O',
 'B-ND06',
 'B-ED09',
 'I-ED09',
 'O',
 'O',
 'O',
 'O',
 'B-ND01',
 'O',
 'B-NC04',
 'O',
 'O',
 'O',
 'O',
 'B-ND01',
 'O',
 'O',
 'B-LC03',
 'O',
 'O',
 'O',
 'B-ND01',
 'B-ND01',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-LC13',
 'O',
 'O',
 'O',
 'B-ND01',
 'B-LC13',
 'O',
 'B-LB17',
 'O',
 'O',
 'O',
 'O',
 'B-SI01',
 'O',
 'B-SF01',
 'O',
 'B-SI01',
 'O',
 'O',
 'B-LC15',
 'B-LC13',
 'O',
 'O',
 'O',
 'O',
 'B-SI01',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-LC04',
 'O',
 'O',
 'O',
 'B-LC07',
 'B-SF01',
 'O',
 'B-LC03',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-LB17',
 'O',
 'B-NC04',
 'B-ND01',
 'O',
 'O',
 'O',
 'O',
 'B-NC04',
 '

In [52]:
report_test_y_code[0]

['[CLS]',
 'O',
 'B-ND01',
 'O',
 'O',
 'B-NC04',
 'B-ED09',
 'I-ED09',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-NC04',
 'O',
 'O',
 'B-EH07',
 'O',
 'O',
 'B-LC03',
 'O',
 'O',
 'O',
 'B-ND01',
 'B-ND11',
 'I-ND11',
 'O',
 'O',
 'B-EI05',
 'I-EI05',
 'B-NB06',
 'O',
 'O',
 'O',
 'B-NB06',
 'B-LB02',
 'I-LB02',
 'O',
 'B-LC04',
 'B-ND11',
 'B-SC09',
 'O',
 'O',
 'B-NC04',
 'O',
 'B-ND06',
 'B-ED09',
 'I-ED09',
 'O',
 'O',
 'O',
 'O',
 'B-ND01',
 'O',
 'B-NC04',
 'O',
 'O',
 'O',
 'O',
 'B-ND01',
 'O',
 'O',
 'B-LC03',
 'O',
 'O',
 'O',
 'B-ND01',
 'B-ND01',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-LC13',
 'O',
 'O',
 'O',
 'B-ND01',
 'B-LC13',
 'O',
 'B-LB17',
 'O',
 'O',
 'O',
 'O',
 'B-SI01',
 'O',
 'B-SF01',
 'O',
 'B-SI01',
 'O',
 'O',
 'B-LC15',
 'B-LC13',
 'O',
 'O',
 'O',
 'O',
 'B-SI01',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-LC04',
 'O',
 'O',
 'O',
 'B-LC07',
 'B-SF01',
 'O',
 'B-LC03',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-LB17',
 'O',
 'B-NC04',
 'B-ND01',
 'O',
 'O',
 'O',
 'O',
 'B-

In [55]:
report_file_name = 'report_NER_code_electra_21.11.03-2.txt'
with open(report_file_name, 'w') as f:
    f.write(classification_report_seqeval(y_true=report_test_y_code, y_pred=report_y_pred_code))

In [52]:
# len(report_test_y_code) # 6241261
report_test_y_code

['B-NC04',
 'B-LA01',
 'B-NC08',
 'B-LB17',
 'I-LB17',
 'O',
 'B-NC05',
 'O',
 'O',
 'O',
 'O',
 'B-ND07',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-NC05',
 'O',
 'O',
 'B-LA03',
 'O',
 'B-LC14',
 'I-LC14',
 'O',
 'O',
 'B-EI03',
 'B-LC13',
 'B-LC13',
 'O',
 'B-NC02',
 'B-SG07',
 'O',
 'O',
 'B-NC02',
 'O',
 'O',
 'O',
 'O',
 'B-LC14',
 'I-LC14',
 'B-LB20',
 'O',
 'O',
 'B-LC14',
 'I-LC14',
 'O',
 'O',
 'B-SI05',
 'O',
 'O',
 'B-NC02',
 'O',
 'B-EI10',
 'O',
 'B-OB02',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-LC03',
 'B-LB17',
 'I-LB17',
 'O',
 'O',
 'O',
 'B-LC13',
 'O',
 'O',
 'B-LC01',
 'O',
 'B-NC04',
 'O',
 'B-LC02',
 'I-LC02',
 'O',
 'O',
 'B-NC10',
 'O',
 'O',
 'O',
 'B-SB12',
 'O',
 'O',
 'B-SH01',
 'O',
 'O',
 'B-LB17',
 'I-LB17',
 'O',
 'B-SB12',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-LC04',
 'O',
 'O',
 'B-LC01',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-LC03',
 'B-SI01',
 'O',
 'B-EE02',
 'O',
 'O',
 'O',
 'B-LB12',
 'O',
 'O',
 'B-SC13',
 'O',
 'B-LB09',
 'O',
 'O',
 'B-

In [53]:
# len(report_y_pred_code) # 6241261
report_y_pred_code

['B-NC04',
 'I-LB17',
 'B-NC08',
 'B-LB17',
 'I-LB17',
 'O',
 'B-NC05',
 'O',
 'O',
 'O',
 'O',
 'B-ND07',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-NC05',
 'O',
 'O',
 'B-LA03',
 'O',
 'B-LC14',
 'I-LC14',
 'O',
 'O',
 'B-EI03',
 'B-LC13',
 'B-LC13',
 'O',
 'B-NC02',
 'B-SG07',
 'O',
 'O',
 'B-NC02',
 'O',
 'O',
 'O',
 'O',
 'B-LC14',
 'I-LC14',
 'B-LB20',
 'O',
 'O',
 'B-LC14',
 'I-LC14',
 'O',
 'O',
 'B-SI05',
 'O',
 'O',
 'B-NC02',
 'O',
 'B-EI10',
 'O',
 'B-OB02',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-LC03',
 'B-LB17',
 'I-LB17',
 'O',
 'O',
 'O',
 'B-LC13',
 'O',
 'O',
 'B-LC01',
 'O',
 'B-NC04',
 'O',
 'B-LC02',
 'I-LC02',
 'O',
 'O',
 'B-NC10',
 'O',
 'O',
 'O',
 'B-SB12',
 'O',
 'O',
 'B-SH01',
 'O',
 'O',
 'B-NC09',
 'O',
 'O',
 'B-SB12',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-LC04',
 'O',
 'O',
 'B-LC01',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-LC03',
 'B-SI01',
 'O',
 'B-EE02',
 'O',
 'O',
 'O',
 'B-LB12',
 'O',
 'O',
 'B-SC13',
 'O',
 'B-LB09',
 'O',
 'O',
 'B-LC03'